In [39]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [40]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')


orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [41]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [42]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre', 'count'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre,count
0,1,Toy Story (1995),Animation|Children's|Comedy,NaN
1,2,Jumanji (1995),Adventure|Children's|Fantasy,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN


In [43]:
#유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [44]:
#유니크한 사용자수
ratings['user_id'].nunique()

6039

In [45]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [46]:
#가장 인기있는 영화
top_movies = pd.merge(ratings, movies, how='left', on='movie_id')
#top_movies.head()

tops = top_movies.sort_values(ascending=False, by=['count_x']).head(30)
tops['title']
#movie_count = ratings.groupby('count')['movie_id'].count()
#movie_count.sort_values(ascending=False).head(30)
#movie_count

0                    One Flew Over the Cuckoo's Nest (1975)
327966                               Midnight Cowboy (1969)
651588    Star Wars: Episode VI - Return of the Jedi (1983)
327977    Horseman on the Roof, The (Hussard sur le toit...
327976                      Who Framed Roger Rabbit? (1988)
651585                                     True Lies (1994)
176729                                          Rudy (1993)
176730                              Schindler's List (1993)
327975                                Bank Dick, The (1940)
327974                                  Bikini Beach (1964)
327973                                   Beach Party (1963)
176734                          Fish Called Wanda, A (1988)
176735                                  Best in Show (2000)
176736                     Phantom of the Opera, The (1943)
176737                                     Duck Soup (1933)
176738                          Being John Malkovich (1999)
327972                                 D

In [47]:
#내가 선호하는 영화 5가지 평점 추가
#1909::X-Files: Fight the Future, The (1998)::Mystery|Sci-Fi|Thriller
#3798::What Lies Beneath (2000)::Thriller
#2012::Back to the Future Part III (1990)::Comedy|Sci-Fi|Western
#1::Toy Story (1995)::Animation|Children's|Comedy
#595::Beauty and the Beast (1991)::Animation|Children's|Musical

my_fav_movie_ids = [1909,3798,2012,1,595]

#ratings.sort_values(ascending=False, by=['user_id']).head(10)

my_movie_list = pd.DataFrame({'user_id':[6041]*5, 'movie_id':my_fav_movie_ids, 'count':5})
ratings = ratings.append(my_movie_list)


In [48]:
ratings.sort_values(ascending=False, by=['user_id']).head(10)

,user_id,movie_id,count,timestamp
4,6041,595,5,NaN
3,6041,1,5,NaN
2,6041,2012,5,NaN
1,6041,3798,5,NaN
0,6041,1909,5,NaN
999991,6040,318,4,956704257.0
999984,6040,2575,4,997453762.0
999985,6040,306,4,957716754.0
999986,6040,3521,5,956715569.0
999988,6040,1921,4,997454464.0
